<a href="https://colab.research.google.com/github/ankit-rathi/AR-Talks/blob/master/Postgres_CRUD.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!sudo apt update
!sudo apt install dirmngr ca-certificates software-properties-common gnupg gnupg2 apt-transport-https curl -y
!curl -fSsL https://www.postgresql.org/media/keys/ACCC4CF8.asc | gpg --dearmor | sudo tee /usr/share/keyrings/postgresql.gpg > /dev/null
!echo 'deb [arch=amd64,arm64,ppc64el signed-by=/usr/share/keyrings/postgresql.gpg] http://apt.postgresql.org/pub/repos/apt/ jammy-pgdg main' | sudo tee /etc/apt/sources.list.d/pgdg.list
!sudo apt update
!sudo apt install postgresql-client-15 postgresql-15 -y
!sudo service postgresql start

Hit:1 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:2 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:3 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:4 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:6 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:7 http://apt.postgresql.org/pub/repos/apt jammy-pgdg InRelease
Hit:8 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:11 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
34 packages can be upgraded. Run 'apt list --upgradable' to see them.
Reading package lists... Done


In [2]:
# Setup a password `postgres` for username `postgres`
!sudo -u postgres psql -U postgres -c "CREATE ROLE root WITH SUPERUSER;"
!sudo -u postgres psql -U postgres -c "ALTER ROLE root WITH LOGIN;"
!sudo -u postgres psql -c "ALTER USER postgres PASSWORD 'postgres'"

ERROR:  role "root" already exists
ALTER ROLE
ALTER ROLE


In [5]:
!sudo -u postgres psql -c "CREATE TABLE cities ( city VARCHAR(255), address VARCHAR(255) );"

CREATE TABLE


In [6]:
#!/usr/bin/env python3
import sys, psycopg2, psycopg2.sql as sql


class Crud:
    def __init__(self, user, password, host, port, dbname, table, primarykey):
        self.user = user
        self.password = password
        self.host = host
        self.port = port
        self.dbname = dbname
        self.table = table
        self.primarykey = primarykey



    def connect(self):
        try:
            connection = psycopg2.connect(
                user = self.user,
                password = self.password,
                host = self.host,
                port = self.port,
                dbname = self.dbname
            )
            cursor = connection.cursor()
            print(
                '------------------------------------------------------------'
                '\n-# PostgreSQL connection & transaction is ACTIVE\n'
                )
        except (Exception, psycopg2.Error) as error :
            print(error, error.pgcode, error.pgerror, sep = '\n')
            sys.exit()
        else:
            self._connection = connection
            self._cursor = cursor
            self._counter = 0


    def _check_connection(self):
        try:
            self._connection
        except AttributeError:
            print('ERROR: NOT Connected to Database')
            sys.exit()


    def _execute(self, query, Placeholder_value = None):
        self._check_connection()
        if Placeholder_value == None or None in Placeholder_value:
            self._cursor.execute(query)
            print( '-# ' + query.as_string(self._connection) + ';\n' )
        else:
            self._cursor.execute(query, Placeholder_value)
            print( '-# ' + query.as_string(self._connection) % Placeholder_value + ';\n' )


    def commit(self):
        self._check_connection()
        self._connection.commit()
        print('-# COMMIT '+ str(self._counter) +' changes\n')
        self._counter = 0


    def close(self, commit = False):
        self._check_connection()
        if commit:
            self.commit()
        else:
            self._cursor.close()
            self._connection.close()
        if self._counter > 0:
            print(
                '-# '+ str(self._counter) +' changes NOT commited  CLOSE connection\n'
                '------------------------------------------------------------\n'
            )
        else:
            print(
                '-# CLOSE connection\n'
                '------------------------------------------------------------\n'
            )


    def insert(self, **column_value):
        insert_query  = sql.SQL("INSERT INTO {} ({}) VALUES ({})").format(
            sql.Identifier(self.table),
            sql.SQL(', ').join( map( sql.Identifier, column_value.keys() ) ),
            sql.SQL(', ').join(sql.Placeholder() * len(column_value.values()))
        )
        record_to_insert = tuple(column_value.values())
        self._execute(insert_query, record_to_insert )
        self._counter += 1


    def insert_many(self, columns, rows):
        insert_query  = sql.SQL("INSERT INTO {} ({}) VALUES ({})").format(
            sql.Identifier(self.table),
            sql.SQL(', ').join( map( sql.Identifier, columns ) ),
            sql.SQL(', ').join(sql.Placeholder() * len(rows[0]))
        )
        for row in rows:
            row = tuple(row)
            self._execute(insert_query, row )
            self._counter += 1


    def select(self, columns, primaryKey_value = None):
        if primaryKey_value == None:
            select_query = sql.SQL("SELECT {} FROM {}").format(
                sql.SQL(',').join(map(sql.Identifier, columns)),
                sql.Identifier(self.table)
            )
            self._execute( select_query )
        else:
            select_query = sql.SQL("SELECT {} FROM {} WHERE {} = {}").format(
                sql.SQL(',').join(map(sql.Identifier, columns)),
                sql.Identifier(self.table),
                sql.Identifier(self.primarykey),
                sql.Placeholder()
            )
            self._execute( select_query, ( primaryKey_value,))
        try:
            selected = self._cursor.fetchall()
        except psycopg2.ProgrammingError as error:
            selected = '# ERROR: ' + str(error)
        else:
            print('-# ' + str(selected) + '\n')
            return selected


    def select_all(self, primaryKey_value = None):
        if primaryKey_value == None:
            select_query = sql.SQL("SELECT * FROM {}").format(sql.Identifier(self.table))
            self._execute( select_query )
        else:
            select_query = sql.SQL("SELECT * FROM {} WHERE {} = {}").format(
                sql.Identifier(self.table),
                sql.Identifier(self.primarykey),
                sql.Placeholder()
            )
            self._execute( select_query, ( primaryKey_value,))
        try:
            selected = self._cursor.fetchall()
        except psycopg2.ProgrammingError as error:
            selected = '# ERROR: ' + str(error)
        else:
            print('-# ' + str(selected) + '\n')
            return selected


    def update(self, column, column_value, primaryKey_value):
        update_query  = sql.SQL("UPDATE {} SET {} = {} WHERE {} = {}").format(
            sql.Identifier(self.table),
            sql.Identifier(column),
            sql.Placeholder(),
            sql.Identifier(self.primarykey),
            sql.Placeholder()
        )
        self._execute(update_query, ( column_value, primaryKey_value))
        self._counter += 1


    def update_multiple_columns(self, columns, columns_value, primaryKey_value):
        update_query  = sql.SQL("UPDATE {} SET ({}) = ({}) WHERE {} = {}").format(
            sql.Identifier(self.table),
            sql.SQL(',').join(map(sql.Identifier, columns)),
            sql.SQL(', ').join(sql.Placeholder() * len(columns_value)),
            sql.Identifier(self.primarykey),
            sql.Placeholder()
        )
        Placeholder_value = list(columns_value)
        Placeholder_value.append(primaryKey_value)
        Placeholder_value = tuple(Placeholder_value)
        self._execute(update_query, Placeholder_value)
        self._counter += 1


    def delete(self, primaryKey_value):
        delete_query  = sql.SQL("DELETE FROM {} WHERE {} = {}").format(
            sql.Identifier(self.table),
            sql.Identifier(self.primarykey),
            sql.Placeholder()
        )
        self._execute(delete_query, ( primaryKey_value,))
        self._counter += 1


    def delete_all(self):
        delete_query  = sql.SQL("DELETE FROM {}").format( sql.Identifier(self.table) )
        self._execute(delete_query)
        self._counter += 1

In [8]:
table = Crud(
    user = 'postgres',
    password = 'postgres',
    host = '127.0.0.1',
    port = '5432',
    dbname = 'postgres',
    table = 'cities',
    primarykey = 'city'
)


table.connect()


table.insert(
    city = 'fayoum',
    address = 'south of cairo'
)


table.insert_many(
    columns = ('city', 'address'),
    rows = [
        ['matrooh', 'north'],
        ['luxor', 'south']
    ]
)


table.commit()


table.select_all()


table.select_all(
    primaryKey_value = 'luxor'
)


table.select(
    columns = ['address'],
    primaryKey_value = 'luxor'
)


table.select(
    columns = ['address']
)


table.update(
    column = 'address',
    column_value = '50 KM south of cairo',
    primaryKey_value = 'fayoum'
)


table.update_multiple_columns(
    columns = ['city', 'address'],
    columns_value = ['qena', 'upperEgypt'],
    primaryKey_value = 'luxor'
)


table.delete(
    primaryKey_value = 'matrooh'
)


table.select_all()


table.delete_all()


table.close("commit")

------------------------------------------------------------
-# PostgreSQL connection & transaction is ACTIVE

-# INSERT INTO "cities" ("city", "address") VALUES (fayoum, south of cairo);

-# INSERT INTO "cities" ("city", "address") VALUES (matrooh, north);

-# INSERT INTO "cities" ("city", "address") VALUES (luxor, south);

-# COMMIT 3 changes

-# SELECT * FROM "cities";

-# [('fayoum', 'south of cairo'), ('matrooh', 'north'), ('luxor', 'south')]

-# SELECT * FROM "cities" WHERE "city" = luxor;

-# [('luxor', 'south')]

-# SELECT "address" FROM "cities" WHERE "city" = luxor;

-# [('south',)]

-# SELECT "address" FROM "cities";

-# [('south of cairo',), ('north',), ('south',)]

-# UPDATE "cities" SET "address" = 50 KM south of cairo WHERE "city" = fayoum;

-# UPDATE "cities" SET ("city","address") = (qena, upperEgypt) WHERE "city" = luxor;

-# DELETE FROM "cities" WHERE "city" = matrooh;

-# SELECT * FROM "cities";

-# [('fayoum', '50 KM south of cairo'), ('qena', 'upperEgypt')]

-# DE

In [12]:
table.connect()


table.insert(
    city = 'fayoum',
    address = 'south of cairo'
)


table.insert_many(
    columns = ('city', 'address'),
    rows = [
        ['matrooh', 'north'],
        ['luxor', 'south']
    ]
)


table.commit()


table.select_all()

------------------------------------------------------------
-# PostgreSQL connection & transaction is ACTIVE

-# INSERT INTO "cities" ("city", "address") VALUES (fayoum, south of cairo);

-# INSERT INTO "cities" ("city", "address") VALUES (matrooh, north);

-# INSERT INTO "cities" ("city", "address") VALUES (luxor, south);

-# COMMIT 3 changes

-# SELECT * FROM "cities";

-# [('fayoum', 'south of cairo'), ('matrooh', 'north'), ('luxor', 'south'), ('fayoum', 'south of cairo'), ('matrooh', 'north'), ('luxor', 'south')]



[('fayoum', 'south of cairo'),
 ('matrooh', 'north'),
 ('luxor', 'south'),
 ('fayoum', 'south of cairo'),
 ('matrooh', 'north'),
 ('luxor', 'south')]

In [13]:
table.select_all()

-# SELECT * FROM "cities";

-# [('fayoum', 'south of cairo'), ('matrooh', 'north'), ('luxor', 'south'), ('fayoum', 'south of cairo'), ('matrooh', 'north'), ('luxor', 'south')]



[('fayoum', 'south of cairo'),
 ('matrooh', 'north'),
 ('luxor', 'south'),
 ('fayoum', 'south of cairo'),
 ('matrooh', 'north'),
 ('luxor', 'south')]